In [5]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

# %pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
# %pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
# %pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# Helper function for visualization.
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

def visualize(h, color):
    z = TSNE(n_components=2).fit_transform(h.detach().cpu().numpy())

    plt.figure(figsize=(10,10))
    plt.xticks([])
    plt.yticks([])

    plt.scatter(z[:, 0], z[:, 1], s=70, c=color, cmap="Set2")
    plt.show()

1.12.0


In [6]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

dataset = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())

print()
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')


Dataset: Cora():
Number of graphs: 1
Number of features: 1433
Number of classes: 7

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Number of nodes: 2708
Number of edges: 10556
Average node degree: 3.90
Number of training nodes: 140
Training node label rate: 0.05
Has isolated nodes: False
Has self-loops: False
Is undirected: True


In [7]:
from torch_geometric.nn import GCNConv
from torch.nn import functional as F

class GCN(torch.nn.Module):
    def __init__(self,hidden_channels):
        super().__init__()
        torch.manual_seed(1234567)
        self.conv1=GCNConv(dataset.num_features,hidden_channels)
        self.conv2=GCNConv(hidden_channels,dataset.num_classes)

    def forward(self,x,edge_index):
        x=self.conv1(x,edge_index)
        x=x.relu()
        x=F.dropout(x,p=0.5,training=self.training)
        x=self.conv2(x,edge_index)
        return x
    
model=GCN(16)
print(model)

GCN(
  (conv1): GCNConv(1433, 16)
  (conv2): GCNConv(16, 7)
)


In [8]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model=GCN(hidden_channels=16)
optimizer=torch.optim.Adam(model.parameters(),lr=0.01,weight_decay=5e-4)
criterion=torch.nn.CrossEntropyLoss()

def train():
    model.train()
    optimizer.zero_grad()
    out=model(data.x,data.edge_index)
    loss=criterion(out[data.train_mask],data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss

def test():
    model.eval()
    out=model(data.x,data.edge_index)
    pred=out.argmax(dim=1)
    test_correct=pred[data.test_mask]==data.y[data.test_mask]
    test_acc=int(test_correct.sum())/int(data.test_mask.sum())
    return test_acc

for epoch in range(1,101):
    loss = train()
    print(f'epoch: {epoch:03d},loss: {loss:.4f}')

<IPython.core.display.Javascript object>

epoch: 001,loss: 1.9465
epoch: 002,loss: 1.9419
epoch: 003,loss: 1.9363
epoch: 004,loss: 1.9290
epoch: 005,loss: 1.9199
epoch: 006,loss: 1.9140
epoch: 007,loss: 1.9079
epoch: 008,loss: 1.8992
epoch: 009,loss: 1.8876
epoch: 010,loss: 1.8764
epoch: 011,loss: 1.8656
epoch: 012,loss: 1.8626
epoch: 013,loss: 1.8460
epoch: 014,loss: 1.8329
epoch: 015,loss: 1.8225
epoch: 016,loss: 1.8167
epoch: 017,loss: 1.7995
epoch: 018,loss: 1.7878
epoch: 019,loss: 1.7716
epoch: 020,loss: 1.7568
epoch: 021,loss: 1.7563
epoch: 022,loss: 1.7342
epoch: 023,loss: 1.7092
epoch: 024,loss: 1.7015
epoch: 025,loss: 1.6671
epoch: 026,loss: 1.6757
epoch: 027,loss: 1.6609
epoch: 028,loss: 1.6355
epoch: 029,loss: 1.6339
epoch: 030,loss: 1.6102
epoch: 031,loss: 1.5964
epoch: 032,loss: 1.5721
epoch: 033,loss: 1.5570
epoch: 034,loss: 1.5445
epoch: 035,loss: 1.5093
epoch: 036,loss: 1.4889
epoch: 037,loss: 1.4776
epoch: 038,loss: 1.4704
epoch: 039,loss: 1.4263
epoch: 040,loss: 1.3972
epoch: 041,loss: 1.3873
epoch: 042,loss: